# Laboratorio 6: PyTorch

- Martínez Ostoa Néstor I.
- Aprendizaje de Máquina
- LCD - IIMAS - UNAM 

---
Utilizar la libreta disponible en https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

1) Cambiar el modelo para utilizar alguno de https://pytorch.org/vision/stable/models.html

2) Ejecutar el código correspondiente cambiando el número de épocas y factor de aprendizaje

3) Reportar el valor de error de entrenamiento encontrado al final del entrenamiento

In [23]:
import torch
import torchvision.models as models
from torchvision import transforms as T
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

## Obtención de datos

Para este laboratorio trabajeremos con el dataset **[FashionMINST](https://github.com/zalandoresearch/fashion-mnist)**. 

Cada muestra de datos contiene lo siguiente:

- Imagen de $28\times 28$ pixeles asociada a una de las 10 clases disponibles
- Clases:

|Etiqueta|Descripción|
|--------|-----------|
|0|T-shirt/top|
|1|Trouser|
|2|Pullover|
|3|Dress|
|4|Coat|
|5|Sandal|
|6|Shirt|
|7|Sneaker|
|8|Bag|
|9|Ankle boot|

In [49]:
normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_rgb = T.Lambda(lambda image: image.convert('RGB'))
resize = T.Resize((224, 224))
my_transform = T.Compose([resize, to_rgb, T.ToTensor(), normalize])

In [50]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=my_transform,
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=my_transform,
)

A continuación, creamos un *wrapper** iterable sobre el datset obtenido previamente. Para esto, pasamos el ```Dataset``` como argumento al ```DataLoader```. Este paso ofrece lo siguiente:

- Batch automático
- *Shuffling*
- Carga de datos multiproceso

In [51]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 3, 224, 224])
Shape of y:  torch.Size([64]) torch.int64


## Modelos

- Alexnet

In [43]:
model = models.alexnet(num_classes=10)

## Optimización de parámetros del modelo

Para entrenar un modelo, necesitamos definar una función de pérdida ```losss``` y un optimizador ```optimizer```

In [58]:
def train(dataloader, model, loss_fn, optimizer, verbose=False):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [62]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

El proceso de **entrenamiento** se realiza sobre varias iteraciones (llamadas *epochs*). Durante cada *epoch*, el modelo aprende los parámetros necesarios para realizar mejores predicciones.

In [63]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295326  [    0/60000]
loss: 2.298383  [ 6400/60000]
loss: 2.291610  [12800/60000]
loss: 2.290938  [19200/60000]
loss: 2.283046  [25600/60000]
loss: 2.289245  [32000/60000]
loss: 2.270028  [38400/60000]
loss: 2.268670  [44800/60000]
loss: 2.248740  [51200/60000]
loss: 2.232937  [57600/60000]
Test Error: 
 Accuracy: 16.2%, Avg loss: 2.213443 

Epoch 2
-------------------------------
loss: 2.220263  [    0/60000]
loss: 2.228171  [ 6400/60000]
loss: 2.140955  [12800/60000]
loss: 2.072320  [19200/60000]
loss: 1.531070  [25600/60000]
loss: 1.382225  [32000/60000]
loss: 1.189878  [38400/60000]
loss: 1.295505  [44800/60000]
loss: 1.129514  [51200/60000]
loss: 0.988413  [57600/60000]
Test Error: 
 Accuracy: 64.1%, Avg loss: 1.037142 

Epoch 3
-------------------------------
loss: 1.133973  [    0/60000]
loss: 1.165417  [ 6400/60000]
loss: 0.845403  [12800/60000]
loss: 1.116840  [19200/60000]
loss: 1.030196  [25600/60000]
loss: 1.012407  [32000/600

## Almacenamiento de modelos entrenados

In [64]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Carga de modelos

In [65]:
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

**Resultados finales**:

- Modelo: Alexnet
- Epochs: 5
- Learning rate: 1e-3
- Accuracy: 74,2%